## Preperation: load dependencies

In [1]:
import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
import $ivy.`com.typesafe.slick::slick:3.3.3`
import $ivy.`com.typesafe.slick::slick-hikaricp:3.3.3`
import $ivy.`com.github.tminglei::slick-pg:0.19.5`
import $ivy.`com.github.tminglei::slick-pg_spray-json:0.19.5`
import $ivy.`org.postgresql:postgresql:42.2.19`
import $ivy.`ch.qos.logback:logback-classic:1.2.3`

// Load .jar
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu_2.13-0.1.jar"))

import $ivy.$                                               

import $ivy.$                                

import $ivy.$                                         

import $ivy.$                                     

import $ivy.$                                                

import $ivy.$                                  

import $ivy.$                                     

// Load .jar


In [2]:
// Smile

import $ivy.`com.github.haifengl::smile-scala:2.6.0`

import scala.language.postfixOps
import org.apache.commons.csv.CSVFormat
import smile._
import smile.util._
import smile.math._
import smile.math.MathEx._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.json._
import smile.interpolation._
import smile.validation._
import smile.association._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.regression._
import smile.feature._
import smile.clustering._
import smile.vq._
import smile.manifold._
import smile.mds._
import smile.sequence._
import smile.projection._
import smile.nlp._
import smile.wavelet._
// Plotting
import smile.plot.swing._
import smile.plot.vega._
import smile.plot.show
import smile.plot.Render._

import $ivy.$                                       


import scala.language.postfixOps

import org.apache.commons.csv.CSVFormat

import smile._

import smile.util._

import smile.math._

import smile.math.MathEx._

import smile.math.kernel._

import smile.math.matrix._

import smile.math.matrix.Matrix._

import smile.math.rbf._

import smile.stat.distribution._

import smile.data._

import smile.data.formula._

import smile.data.measure._

import smile.data.`type`._

import smile.json._

import smile.interpolation._

import smile.validation._

import smile.association._

import smile.base.cart.SplitRule

import smile.base.mlp._

import smile.base.rbf.RBF

import smile.classification._

import smile.regression._

import smile.feature._

import smile.clustering._

import smile.vq._

import smile.manifold._

import smile.mds._

import smile.sequence._

import smile.projection._

import smile.nlp._

import smile.wavelet._
// Plotting

import smile.plot.swing._

import smile.plot.vega._

i

## Get data from DB

In [3]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.action.ActionDB


val workflowListDB = new WorkflowListDB()
val actionDB = new ActionDB()

val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val actions =  Await.result(actionDB.getActions, 2 seconds)

09:49:40.548 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding uuid -> java.util.UUID 
09:49:40.554 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding text -> java.lang.String 
09:49:40.554 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding bool -> Boolean 
09:49:40.648 [scala-interpreter-1] INFO  com.zaxxer.hikari.HikariDataSource - workflow.postgres.db - Starting...
09:49:40.678 [scala-interpreter-1] INFO  com.zaxxer.hikari.HikariDataSource - workflow.postgres.db - Start completed.


import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

import scala.concurrent.duration._

import scala.concurrent.{ Await, Future }

import scala.language.postfixOps


import de.tmrdlt.database.workflowlist.WorkflowListDB

import de.tmrdlt.database.action.ActionDB



workflowListDB: WorkflowListDB = de.tmrdlt.database.workflowlist.WorkflowListDB@1cfb1054
actionDB: ActionDB = de.tmrdlt.database.action.ActionDB@1483729
workflowLists: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-08T11:11:54.658
  ),
  WorkflowList(
    2L,
    "5fad0555e165558a64ad0479",
    "Tasks / ToDos",
    None,
    Some(1L),
    0L,
    LIST,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-08T11:11:54.658
  ),
  WorkflowList(
    3L,
    "5fad0555

## Size of Board over time

In [4]:
import de.tmrdlt.models._
import java.time.LocalDate
import scala.collection.mutable.ListBuffer

val boards = workflowLists.filter(_.listType == WorkflowListType.BOARD)
val board = boards.head

val data = new ListBuffer[(Int, LocalDate)]
data += ((0, board.createdAt.toLocalDate))

val sortedActions = actions
    .filter(a => a.actionType == "createCard" || a.actionType == "deleteCard")
    .sortBy(_.date)

sortedActions.foreach{ a => 
    val count = if (a.actionType == "createCard") data.last._1 + 1 else data.last._1 - 1
    data += ((count, a.date.toLocalDate))
}


import de.tmrdlt.models._

import java.time.LocalDate

import scala.collection.mutable.ListBuffer


boards: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-08T11:11:54.658
  )
)
board: de.tmrdlt.database.workflowlist.WorkflowList = WorkflowList(
  1L,
  "5fad0555e165558a64ad0478",
  "Masterarbeit",
  Some(""),
  None,
  0L,
  BOARD,
  Some(OPEN),
  Trello,
  None,
  2020-11-12T10:50:13,
  2021-03-08T11:11:54.658
)
data: ListBuffer[(Int, LocalDate)] = ListBuffer(
  (0, 2020-11-12),
  (1, 2020-11-12),
  (2, 2020-11-12),
  (3, 2020-11-12),
  (4, 2020-11-12),
  (5, 2020-11-30),
  (6, 2020-11-30),
  (7, 2020-11-30),
  (8, 2020-11-30),
  (9, 2020-11-30),
  (10, 2020-12-01),
  (11, 2020-12-01),
  (12, 2020-12-01),
  (13, 2020-12-01),
  (14, 2020-12-04),
  (15, 2020-12-04),
  (14, 2020-12-07

In [26]:
val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
val chart = VegaLite.view().
    mark(JsObject("type" -> "area", "line" -> true, "point" -> true)).
    width(800).
    height(200).
    x(field = "date", `type` = "temporal").
    y(field = "count", `type` = "quantitative").
    data(smileData).
    title(s"Size of the board '${board.title}' over time")

show(chart)

<iframe id="2ea28bee-5502-454f-806c-8c59bee64c0e" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-11-12","count":1},{"date":"2020-11-12","count":2},{"date":"2020-11-12","count":3},{"date":"2020-11-12","count":4},{"date":"2020-11-30","count":5},{"date":"2020-11-30","count":6},{"date":"2020-11-30","count":7},{"date":"2020-11-30","count":8},{"date":"2020-11-30","count":9},{"date":"2020-12-01","count":10},{"date":"2020-12-01","count":11},{"date":"2020-12-01","count":12},{"date":"2020-12-01","count":13},{"date":"2020-12-04","count":14},{"date":"2020-12-04","count":15},{"date":"2020-12-07","count":14},{"date":"2020-12-07","count":13},{"date":"2020-12-07","count":14},{"date":"2020-12-08","count":15},{"date":"2020-12-14","count":16},{"date":"2021-01-25","count":17},{"date":"2021-02-02","count":18},{"date":"2021-02-02","count":19},{"date":"2021-02-02","count":20},{"date":"2021-02-09","count":21},{"date":"2021-02-15","count":22},{"date":"2021-02-15","count":23},{"date":"2021-02-22","count":24},{"date":"2021-02-23","count":25},{"date":"2021-02-26","count":26},{"date":"2021-03-08","count":27},{"date":"2021-03-08","count":28},{"date":"2020-11-12","count":0}]},"width":800,"title":"Size of the board 'Masterarbeit' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

smileData: List[JsObject] = List(
  JsObject(HashMap("date" -> JsDate(2020-11-12), "count" -> JsInt(0))),
  JsObject(HashMap("date" -> JsDate(2020-11-12), "count" -> JsInt(1))),
  JsObject(HashMap("date" -> JsDate(2020-11-12), "count" -> JsInt(2))),
  JsObject(HashMap("date" -> JsDate(2020-11-12), "count" -> JsInt(3))),
  JsObject(HashMap("date" -> JsDate(2020-11-12), "count" -> JsInt(4))),
  JsObject(HashMap("date" -> JsDate(2020-11-30), "count" -> JsInt(5))),
  JsObject(HashMap("date" -> JsDate(2020-11-30), "count" -> JsInt(6))),
  JsObject(HashMap("date" -> JsDate(2020-11-30), "count" -> JsInt(7))),
  JsObject(HashMap("date" -> JsDate(2020-11-30), "count" -> JsInt(8))),
  JsObject(HashMap("date" -> JsDate(2020-11-30), "count" -> JsInt(9))),
  JsObject(HashMap("date" -> JsDate(2020-12-01), "count" -> JsInt(10))),
  JsObject(HashMap("date" -> JsDate(2020-12-01), "count" -> JsInt(11))),
  JsObject(HashMap("date" -> JsDate(2020-12-01), "count" -> JsInt(12))),
  JsObject(HashMap("date" -

## Size of columns over time

In [27]:
val columns = workflowLists
    .filter(_.listType == WorkflowListType.LIST)
    .filter(_.parentId == Some(board.id))

columns.foreach { column =>
    val data = new ListBuffer[(Int, LocalDate)]
    data += ((0, column.createdAt.toLocalDate))
    
    
    val relevantActions = actions
        .filter(a => a.parentApiId == Some(column.apiId) ||
                     a.oldParentApiId == Some(column.apiId) ||
                     a.newParentApiId == Some(column.apiId))
        .sortBy(_.date)
    
    relevantActions.foreach{ a => 
        val count = 
            if (a.actionType == "createCard") {
                data.last._1 + 1
            } else if (a.actionType == "deleteCard") {
                data.last._1 - 1  
            } else {
                if (column.apiId == a.oldParentApiId.get) {
                    data.last._1 - 1
                } else {
                    data.last._1 + 1
                }
            } 
        data += ((count, a.date.toLocalDate))
    }
    
    val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
    val chart = VegaLite.view().
        mark(JsObject("type" -> "area", "line" -> true, "point" -> true)).
        width(800).
        height(200).
        x(field = "date", `type` = "temporal").
        y(field = "count", `type` = "quantitative").
        data(smileData).
        title(s"Size of column '${column.title}' over time")
    show(chart)
}


<iframe id="b129c92b-edc4-4eee-920a-d96de4be46be" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-11-12","count":1},{"date":"2020-11-12","count":2},{"date":"2020-11-12","count":3},{"date":"2020-11-12","count":4},{"date":"2020-11-30","count":5},{"date":"2020-11-30","count":4},{"date":"2020-11-30","count":3},{"date":"2020-11-30","count":2},{"date":"2020-11-30","count":3},{"date":"2020-11-30","count":4},{"date":"2020-12-01","count":5},{"date":"2020-12-01","count":6},{"date":"2020-12-01","count":5},{"date":"2020-12-01","count":4},{"date":"2020-12-04","count":5},{"date":"2020-12-04","count":4},{"date":"2020-12-04","count":3},{"date":"2020-12-04","count":2},{"date":"2020-12-07","count":1},{"date":"2020-12-07","count":0},{"date":"2020-12-07","count":-1},{"date":"2020-12-07","count":0},{"date":"2020-12-08","count":1},{"date":"2020-12-08","count":0},{"date":"2020-12-14","count":1},{"date":"2020-12-14","count":2},{"date":"2020-12-14","count":3},{"date":"2020-12-14","count":4},{"date":"2020-12-14","count":3},{"date":"2020-12-14","count":4},{"date":"2020-12-14","count":5},{"date":"2020-12-14","count":6},{"date":"2021-01-25","count":7},{"date":"2021-02-09","count":8},{"date":"2021-02-15","count":9},{"date":"2021-02-15","count":8},{"date":"2021-02-22","count":9},{"date":"2021-02-26","count":10},{"date":"2021-02-26","count":9},{"date":"2021-03-08","count":10}]},"width":800,"title":"Size of column 'Tasks / ToDos' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="e7c3f7b1-420e-4066-845c-d7e4bfe7aeed" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-11-30","count":1},{"date":"2020-11-30","count":2},{"date":"2020-11-30","count":3},{"date":"2020-12-01","count":4},{"date":"2020-12-01","count":5},{"date":"2020-12-01","count":4},{"date":"2020-12-01","count":5},{"date":"2020-12-01","count":4},{"date":"2020-12-01","count":5},{"date":"2020-12-01","count":6},{"date":"2020-12-01","count":7},{"date":"2020-12-04","count":8},{"date":"2020-12-04","count":9},{"date":"2020-12-04","count":8},{"date":"2020-12-08","count":7},{"date":"2020-12-08","count":8},{"date":"2020-12-14","count":7},{"date":"2020-12-14","count":6},{"date":"2020-12-14","count":5},{"date":"2020-12-14","count":4},{"date":"2020-12-14","count":3},{"date":"2020-12-14","count":2},{"date":"2020-12-14","count":1},{"date":"2020-12-14","count":0},{"date":"2020-12-14","count":1},{"date":"2021-01-19","count":0},{"date":"2021-01-19","count":1},{"date":"2021-02-02","count":2},{"date":"2021-02-02","count":1},{"date":"2021-02-15","count":2},{"date":"2021-02-15","count":1}]},"width":800,"title":"Size of column 'Im Gange' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="65cc64fc-ed0f-4dd2-bc23-4d9c8d3d055e" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-12-01","count":1},{"date":"2020-12-01","count":0},{"date":"2020-12-01","count":1},{"date":"2020-12-01","count":0},{"date":"2020-12-04","count":1},{"date":"2020-12-04","count":0},{"date":"2020-12-04","count":1},{"date":"2020-12-07","count":2},{"date":"2020-12-07","count":1},{"date":"2020-12-07","count":2},{"date":"2020-12-08","count":3},{"date":"2020-12-14","count":4},{"date":"2020-12-14","count":5},{"date":"2020-12-14","count":6},{"date":"2020-12-14","count":5},{"date":"2021-01-19","count":6},{"date":"2021-01-19","count":5},{"date":"2021-02-02","count":4},{"date":"2021-02-15","count":5}]},"width":800,"title":"Size of column 'Fertig' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="173d1efd-8ef2-40d6-88f1-d8fa4d21ff85" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-30","count":0},{"date":"2020-11-30","count":1},{"date":"2020-11-30","count":2},{"date":"2020-12-04","count":3},{"date":"2020-12-04","count":4},{"date":"2021-02-02","count":3}]},"width":800,"title":"Size of column 'Resources' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="d3a22f66-f651-430a-85a8-d6db725354bd" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2021-02-02","count":0},{"date":"2021-02-02","count":1},{"date":"2021-02-02","count":2},{"date":"2021-02-02","count":3},{"date":"2021-02-02","count":4},{"date":"2021-02-02","count":5},{"date":"2021-02-15","count":6},{"date":"2021-02-23","count":7},{"date":"2021-03-08","count":8}]},"width":800,"title":"Size of column 'Treffen' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="dcdaa2e5-435b-4c59-9b7c-cd1afae455de" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2021-02-19","count":0},{"date":"2021-02-26","count":1}]},"width":800,"title":"Size of column 'Test' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

columns: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    2L,
    "5fad0555e165558a64ad0479",
    "Tasks / ToDos",
    None,
    Some(1L),
    0L,
    LIST,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-08T11:11:54.658
  ),
  WorkflowList(
    3L,
    "5fad0555e165558a64ad047a",
    "Im Gange",
    None,
    Some(1L),
    1L,
    LIST,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-02-19T15:16:15.684
  ),
  WorkflowList(
    4L,
    "5fad0555e165558a64ad047b",
    "Fertig",
    None,
    Some(1L),
    2L,
    LIST,
    Some(OPEN),
    Trello,
...